In [2]:
# PySpark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, regexp_extract
from pyspark.sql.types import FloatType, StringType, VectorUDT
from pyspark.sql.window import Window

# PySpark MLlib imports
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    VectorAssembler, StringIndexer, OneHotEncoder, Tokenizer, 
    HashingTF, IDF, PolynomialExpansion, StopWordsRemover
)

# Spark NLP imports
from sparknlp.base import *
from sparknlp.annotator import (
    Tokenizer as NLPTokenizer, ViveknSentimentModel, BertSentenceEmbeddings
)
from sparknlp.pretrained import PretrainedPipeline

# Standard MLflow imports
import mlflow

# Scikit-Learn imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

# XGBoost and LightGBM imports
import xgboost as xgb
import lightgbm as lgb

# Standard libraries
import numpy as np
import time
import itertools


# Start Spark session with appropriate parallelism
spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.memoryOverhead", "1g") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.default.parallelism", "4") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "2") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.0") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

df = spark.read.parquet('processed_data.parquet')

In [7]:
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.ml.functions import vector_to_array
def array_to_dense_vector(array):
    return Vectors.dense(array) if array else Vectors.dense([])

array_to_vector_udf = F.udf(array_to_dense_vector, VectorUDT())
df = df.withColumn("author_publisher_combined_embedding", array_to_vector_udf(F.flatten("author_publisher_combined_embedding")))
columns_to_drop = [
    "authors",
    "publisher",
    "published_era",
    "categories"
]

# Drop the columns
df_cleaned = df.drop(*columns_to_drop)

dense_vector_udf = F.udf(lambda v: Vectors.dense(v.toArray()) if v is not None else None, VectorUDT())

df = df.withColumn("title_tfidf_dense", dense_vector_udf(F.col("title_tfidf")))
df = df.withColumn("desc_tfidf_dense", dense_vector_udf(F.col("desc_tfidf")))
df = df.drop("title_tfidf", "desc_tfidf")
df.printSchema()
# Define final feature columns
feature_columns = [
    "publishedYear",
    "book_age",
    "published_decade",
    "published_century",
    "author_frequency",
    "publisher_frequency",
    "published_era_encoded",
    "Category_Index",
    "title_sentiment_encoded",
    "description_sentiment_encoded",
    "title_tfidf_dense",
    "desc_tfidf_dense",
    "author_publisher_combined_embedding"
]

# Filter out any columns that do not exist in the DataFrame
available_feature_columns = [col for col in feature_columns if col in df.columns]

# # Assemble all available features into a single vector column
assembler = VectorAssembler(inputCols=available_feature_columns, outputCol="assembled_features")
df_assembled = assembler.transform(df)

root
 |-- authors: string (nullable = false)
 |-- publisher: string (nullable = true)
 |-- Title: string (nullable = false)
 |-- description: string (nullable = false)
 |-- categories: string (nullable = true)
 |-- Impact: double (nullable = true)
 |-- publishedYear: integer (nullable = true)
 |-- book_age: integer (nullable = true)
 |-- published_decade: integer (nullable = true)
 |-- published_century: integer (nullable = true)
 |-- published_era: string (nullable = false)
 |-- title_tfidf: vector (nullable = true)
 |-- desc_tfidf: vector (nullable = true)
 |-- published_era_index: double (nullable = false)
 |-- published_era_encoded: vector (nullable = true)
 |-- author_frequency: long (nullable = true)
 |-- publisher_frequency: long (nullable = true)
 |-- author_publisher_combined_embedding: vector (nullable = true)
 |-- Category_Index: vector (nullable = true)
 |-- title_sentiment_encoded: integer (nullable = false)
 |-- description_sentiment_encoded: integer (nullable = false)

r

# Model Selection

In [8]:
# Split the data
train_df, val_df, test_df = df_assembled.randomSplit([0.7, 0.15, 0.15], seed=42)

# Convert features to array type for compatibility
train_df = train_df.withColumn("assembled_features_array", vector_to_array(col("assembled_features")))
val_df = val_df.withColumn("assembled_features_array", vector_to_array(col("assembled_features")))
test_df = test_df.withColumn("assembled_features_array", vector_to_array(col("assembled_features")))

# Extract data for LightGBM
X_train = np.array(train_df.select("assembled_features_array").collect()).squeeze()
y_train = np.array(train_df.select("Impact").collect()).squeeze()
X_val = np.array(val_df.select("assembled_features_array").collect()).squeeze()
y_val = np.array(val_df.select("Impact").collect()).squeeze()
X_test = np.array(test_df.select("assembled_features_array").collect()).squeeze()
y_test = np.array(test_df.select("Impact").collect()).squeeze()


In [16]:
mlflow.set_experiment("Model Selections")

2024/11/07 10:10:41 INFO mlflow.tracking.fluent: Experiment with name 'Model Selections' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/624289668838002876', creation_time=1730974241949, experiment_id='624289668838002876', last_update_time=1730974241949, lifecycle_stage='active', name='Model Selections', tags={}>

In [17]:
# Define LightGBM parameters
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 10,
    'random_state': 42
}

# Start MLflow experiment
mlflow.set_experiment("LightGBM Model")

with mlflow.start_run(run_name="LightGBM_CV"):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores, mape_scores = [], []
    rmse_train_scores, mape_train_scores = [], []
    feature_importance_agg = np.zeros(X_train.shape[1])

    # Track training time
    start_time = time.time()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Initialize and train the model
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)
        model = lgb.train(lgb_params, train_data, valid_sets=[val_data])

        # Training predictions and metrics for this fold
        y_train_pred = model.predict(X_train_fold)
        rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
        rmse_train_scores.append(rmse_train)
        mape_train_scores.append(mape_train)

        # Predictions and metrics for this fold
        y_val_pred = model.predict(X_val_fold)
        rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

        # Aggregate feature importances
        feature_importance_agg += model.feature_importance(importance_type="gain")

    # Training time
    training_time = time.time() - start_time

    # Log metrics and parameters
    mlflow.log_param("learning_rate", lgb_params['learning_rate'])
    mlflow.log_param("max_depth", lgb_params['max_depth'])
    mlflow.log_param("n_estimators", lgb_params['n_estimators'])
    mlflow.log_metric("rmse_cv", np.mean(rmse_scores))
    mlflow.log_metric("mape_cv", np.mean(mape_scores))
    mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
    mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))

    # Feature importance
    feature_importance = feature_importance_agg / kf.get_n_splits()
    mlflow.log_param("feature_importance", dict(enumerate(feature_importance)))

    print(f"Training completed - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)} RMSE train: {np.mean(rmse_train_scores)}, MAPE train: {np.mean(mape_train_scores)}")


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.654520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69014
[LightGBM] [Info] Number of data points in the train set: 77747, number of used features: 326
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.715988 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69041
[LightGBM] [Info] Number of data points in the train set: 77747, number of used features: 326
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.610933 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69017
[LightGBM] [Info] Number of data points in the train set: 77747, number of used features: 326
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.657201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69016
[LightGBM] [Info] Number of data points in the train set: 77748, number of used features: 326
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
# Define XGBoost parameters
xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 10,
    'eval_metric': 'rmse',
    'random_state':42
}
# Start MLflow experiment
mlflow.set_experiment("XGBoost Model")

with mlflow.start_run(run_name="XGBoost_CV"):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores, mape_scores = [], []
    rmse_train_scores, mape_train_scores = [], []
    feature_importance_agg = np.zeros(X_train.shape[1])

    # Track training time
    start_time = time.time()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Initialize and train the model
        model = xgb.XGBRegressor(**xgb_params)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], verbose=False)

        # Training predictions and metrics for this fold
        y_train_pred = model.predict(X_train_fold)
        rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
        rmse_train_scores.append(rmse_train)
        mape_train_scores.append(mape_train)
        # Predictions and metrics for this fold
        y_val_pred = model.predict(X_val_fold)
        rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

        # Aggregate feature importances
        feature_importance_agg += model.feature_importances_

    # Training time
    training_time = time.time() - start_time

    # Log metrics and parameters
    mlflow.log_param("learning_rate", xgb_params['learning_rate'])
    mlflow.log_param("max_depth", xgb_params['max_depth'])
    mlflow.log_param("n_estimators", xgb_params['n_estimators'])
    mlflow.log_metric("rmse_cv", np.mean(rmse_scores))
    mlflow.log_metric("mape_cv", np.mean(mape_scores))
    mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
    mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))

    # Feature importance
    feature_importance = feature_importance_agg / kf.get_n_splits()
    mlflow.log_param("feature_importance", dict(enumerate(feature_importance)))

    print(f"Training completed - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)} RMSE train: {np.mean(rmse_train_scores)}, MAPE train: {np.mean(mape_train_scores)}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

Training completed - RMSE CV: 0.2723380424942968, MAPE CV: 244950474942.54712 RMSE train: 0.2711322116207555, MAPE train: 226596335949.44946


In [19]:
# Define Decision Tree parameters
dt_params = {
    'max_depth': 10,
    'min_samples_split': 5,
    'random_state': 42
}

# Start MLflow experiment
mlflow.set_experiment("Decision Tree Model")

with mlflow.start_run(run_name="DecisionTree_CV"):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []  # Corrected: initialized as individual lists
    mape_scores = []  # Corrected: initialized as individual lists
    rmse_train_scores, mape_train_scores = [], []

    # Track training time
    start_time = time.time()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Initialize and train the model
        model = DecisionTreeRegressor(**dt_params)
        model.fit(X_train_fold, y_train_fold)
        # Training predictions and metrics for this fold
        y_train_pred = model.predict(X_train_fold)
        rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
        rmse_train_scores.append(rmse_train)
        mape_train_scores.append(mape_train)
        # Predictions and metrics for this fold
        y_val_pred = model.predict(X_val_fold)
        rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Training time
    training_time = time.time() - start_time

    # Log metrics and parameters
    mlflow.log_param("max_depth", dt_params['max_depth'])
    mlflow.log_param("min_samples_split", dt_params['min_samples_split'])
    mlflow.log_metric("rmse_cv", np.mean(rmse_scores))
    mlflow.log_metric("mape_cv", np.mean(mape_scores))
    mlflow.log_metric("training_time", training_time)
    mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
    mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))

    print(f"Training completed - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)} RMSE train: {np.mean(rmse_train_scores)}, MAPE train: {np.mean(mape_train_scores)}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

Training completed - RMSE CV: 0.2756366994079403, MAPE CV: 244485444975.64517 RMSE train: 0.261037821453372, MAPE train: 178763921900.99408


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [20]:
# Function to train a model with cross-validation and log metrics
def train_model_with_cv(model, model_name):
    with mlflow.start_run(run_name=f"{model_name}_CV"):
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores, mape_scores = [], []
        rmse_train_scores, mape_train_scores = [], []
        # Track training time
        start_time = time.time()

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            # Train the model on the fold
            model.fit(X_train_fold, y_train_fold)

            # Training predictions and metrics for this fold
            y_train_pred = model.predict(X_train_fold)
            rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
            mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
            rmse_train_scores.append(rmse_train)
            mape_train_scores.append(mape_train)
            # Predictions and metrics for this fold
            y_val_pred = model.predict(X_val_fold)
            rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
            mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
            rmse_scores.append(rmse)
            mape_scores.append(mape)

        # Training time
        training_time = time.time() - start_time

        # Log metrics and parameters to MLflow
        mlflow.log_metric("rmse_cv", np.mean(rmse_scores))
        mlflow.log_metric("mape_cv", np.mean(mape_scores))
        mlflow.log_metric("training_time", training_time)
        mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
        mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))
        print(f"{model_name} - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)}  RMSE train: {np.mean(rmse_train_scores)}, MAPE train: {np.mean(mape_train_scores)}")

# Initialize models with configurations
linear_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)  # You can adjust alpha for Ridge regularization strength
lasso_model = Lasso(alpha=0.1)  # You can adjust alpha for Lasso regularization strength

# Train and log each model
train_model_with_cv(linear_model, "LinearRegression")
train_model_with_cv(ridge_model, "RidgeRegression")
train_model_with_cv(lasso_model, "LassoRegression")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

LinearRegression - RMSE CV: 0.2728200299716571, MAPE CV: 242985366089.22647  RMSE train: 0.27204532681410376, MAPE train: 242588494509.6242


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

RidgeRegression - RMSE CV: 0.2728095199709154, MAPE CV: 243131019642.20865  RMSE train: 0.27205234064311623, MAPE train: 242703547423.73126


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

LassoRegression - RMSE CV: 0.27864331214264837, MAPE CV: 246907122340.8542  RMSE train: 0.2786456480827999, MAPE train: 246923185760.18823


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
# Define Random Forest parameters
rf_params = {
    'n_estimators': 10,
    'max_depth': 5,
    'min_samples_split': 5,
    'random_state': 42
}

# Start MLflow experiment
mlflow.set_experiment("Random Forest Model with Cross-Validation")

with mlflow.start_run(run_name="RandomForest_CV"):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores, mape_scores = [], []
    rmse_train_scores, mape_train_scores = [], []
    feature_importance_agg = np.zeros(X_train.shape[1])

    # Track training time
    start_time = time.time()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Initialize and train the model
        model = RandomForestRegressor(**rf_params)
        model.fit(X_train_fold, y_train_fold)

        # Training predictions and metrics for this fold
        y_train_pred = model.predict(X_train_fold)
        rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
        rmse_train_scores.append(rmse_train)
        mape_train_scores.append(mape_train)

        # Predictions and metrics for this fold
        y_val_pred = model.predict(X_val_fold)
        rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

        # Aggregate feature importances
        feature_importance_agg += model.feature_importances_

    # Training time
    training_time = time.time() - start_time

    # Log metrics and parameters
    mlflow.log_param("n_estimators", rf_params['n_estimators'])
    mlflow.log_param("max_depth", rf_params['max_depth'])
    mlflow.log_param("min_samples_split", rf_params['min_samples_split'])
    mlflow.log_metric("rmse_cv", np.mean(rmse_scores))
    mlflow.log_metric("mape_cv", np.mean(mape_scores))
    mlflow.log_metric("training_time", training_time)
    mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
    mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))

    # Average feature importance across folds
    feature_importance = feature_importance_agg / kf.get_n_splits()
    mlflow.log_param("feature_importance", dict(enumerate(feature_importance)))

    print(f"Training completed - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)} RMSE train: {np.mean(rmse_train_scores)}, MAPE train: {np.mean(mape_train_scores)}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

Training completed - RMSE CV: 0.2732579284172062, MAPE CV: 245212722344.48138 RMSE train: 0.27194544181941616, MAPE train: 245333626358.1103


In [22]:
!zip -r /content/mlruns.zip /content/mlruns

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/624289668838002876/ (stored 0%)
  adding: content/mlruns/624289668838002876/meta.yaml (deflated 30%)
  adding: content/mlruns/564800884828740230/ (stored 0%)
  adding: content/mlruns/564800884828740230/meta.yaml (deflated 30%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/ (stored 0%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/meta.yaml (deflated 44%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/params/ (stored 0%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/artifacts/ (stored 0%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/tags/ (stored 0%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/tags/mlflow.source.name (deflated 5%)
  adding: content/mlruns/564800884828740230/9244ebea469d4550a862bab3e61f44d1/tags/mlflow.user (stored 0%)
  add

# Feature Selections

In [74]:
# Define LightGBM parameters
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 10,
    'random_state': 42,
    'num_leaves': 31,
}

# Feature columns before assembling
feature_columns = [
    "publishedYear", "book_age", "published_decade", "published_century",
    "author_frequency", "publisher_frequency", "published_era_encoded",
    "Category_Index", "title_sentiment_encoded", "description_sentiment_encoded",
    "title_tfidf_dense", "desc_tfidf_dense", "author_publisher_combined_embedding"
]

# Start MLflow experiment
mlflow.set_experiment("LightGBM Model with CV Feature Importance")

with mlflow.start_run(run_name="LightGBM_CV"):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse_scores, mape_scores = [], []
    rmse_train_scores, mape_train_scores = [], []

    # Placeholder for feature importances
    feature_importance_agg = np.zeros(len(feature_columns))

    # Track training time
    start_time = time.time()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Initialize and train the model
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)
        model = lgb.train(lgb_params, train_data, valid_sets=[val_data])

        # Training predictions and metrics for this fold
        y_train_pred = model.predict(X_train_fold)
        rmse_train = mean_squared_error(y_train_fold, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train_fold, y_train_pred)
        rmse_train_scores.append(rmse_train)
        mape_train_scores.append(mape_train)

        # Validation predictions and metrics for this fold
        y_val_pred = model.predict(X_val_fold)
        rmse_val = mean_squared_error(y_val_fold, y_val_pred, squared=False)
        mape_val = mean_absolute_percentage_error(y_val_fold, y_val_pred)
        rmse_scores.append(rmse_val)
        mape_scores.append(mape_val)

        # Aggregate feature importances
        fold_importance = model.feature_importance(importance_type="gain")
        feature_importance_agg += fold_importance[:len(feature_columns)]  # Ensure alignment with `feature_columns`

        print(f"Fold {fold + 1} - RMSE train: {rmse_train}, MAPE train: {mape_train}, RMSE val: {rmse_val}, MAPE val: {mape_val}")

    # Calculate the average feature importance across folds
    avg_feature_importance = feature_importance_agg / kf.get_n_splits()
    feature_importance_dict = {feature_columns[i]: avg_feature_importance[i] for i in range(len(feature_columns))}

    # Training time
    total_training_time = time.time() - start_time
    avg_training_time = total_training_time / kf.get_n_splits()

    # Log average metrics across folds and feature importance to MLflow
    mlflow.log_metric("rmse_train_cv", np.mean(rmse_train_scores))
    mlflow.log_metric("mape_train_cv", np.mean(mape_train_scores))
    mlflow.log_metric("rmse_val_cv", np.mean(rmse_scores))
    mlflow.log_metric("mape_val_cv", np.mean(mape_scores))
    mlflow.log_metric("avg_training_time_per_fold", avg_training_time)
    mlflow.log_param("feature_importance", feature_importance_dict)

    print(f"Training completed - RMSE CV: {np.mean(rmse_scores)}, MAPE CV: {np.mean(mape_scores)}, RMSE train CV: {np.mean(rmse_train_scores)}, MAPE train CV: {np.mean(mape_train_scores)}")
    print("Feature Importance:", feature_importance_dict)


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.611293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69066
[LightGBM] [Info] Number of data points in the train set: 87465, number of used features: 326
[LightGBM] [Info] Start training from score 5.328565
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 1 - RMSE train: 0.271838030000925, MAPE train: 268228462209.159, RMSE val: 0.2684144252475, MAPE val: 0.035028060329213705
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.768882 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69060
[LightGBM] [Info] Number of data points in the train set: 87465, number of used features: 326
[LightGBM] [Info] Start training from score 5.328447
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 2 - RMSE train: 0.27148270618963805, MAPE train: 269758174807.3131, RMSE val: 0.27297460527782075, MAPE val: 0.03582248861250693


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 8 - RMSE train: 0.27166147885259706, MAPE train: 266842133557.74097, RMSE val: 0.2721467776096827, MAPE val: 0.03543593714119605
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.595148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69071
[LightGBM] [Info] Number of data points in the train set: 87466, number of used features: 326
[LightGBM] [Info] Start training from score 5.328170
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fold 9 - RMSE train: 0.2714804741324342, MAPE train: 0.03550780871620538, RMSE val: 0.2750197339791205, MAPE val: 2455635574944.195
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.541667 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69062
[LightGBM] [Info] Number of data points in the train set: 87466, number of used features: 326
[LightGBM] [Info] Start training from score 5.327990
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 10 - RMSE train: 0.2717078068052201, MAPE train: 267183497280.22147, RMSE val: 0.27255743966187806, MAPE val: 0.03542494070932278
Training completed - RMSE CV: 0.27256446602733364, MAPE CV: 245563557494.4515, RMSE train CV: 0.2716136902659271, MAPE train CV: 240472745037.8199
Feature Importance: {'publishedYear': 24.844626356288792, 'book_age': 0.0, 'published_decade

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [76]:
mlflow.set_experiment("Feature Selections")

<Experiment: artifact_location='file:///content/mlruns/185631005378406107', creation_time=1730974982241, experiment_id='185631005378406107', last_update_time=1730974982241, lifecycle_stage='active', name='Feature Selections', tags={}>

# Hyper Parameter Tuning

In [84]:
# Define hyperparameter search space
param_space = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'num_leaves': [20, 31, 50],
    'n_estimators': [50, 100, 200]
}

mlflow.set_experiment("LightGBM Grid Search Hyperparameter Tuning")

# Function to train LightGBM and log metrics
def train_and_log_lightgbm(params):
    with mlflow.start_run(nested=True):
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        # Train LightGBM model
        model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, val_data]
        )

        # Training metrics
        y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
        rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
        mape_train = mean_absolute_percentage_error(y_train, y_train_pred)

        # Validation metrics
        y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
        rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
        mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

        # Log parameters and metrics to MLflow
        mlflow.log_params(params)
        mlflow.log_metric("rmse_train", rmse_train)
        mlflow.log_metric("mape_train", mape_train)
        mlflow.log_metric("rmse_val", rmse_val)
        mlflow.log_metric("mape_val", mape_val)

        print(f"RMSE Train: {rmse_train}, MAPE Train: {mape_train}, RMSE Val: {rmse_val}, MAPE Val: {mape_val}")

        return rmse_val

# Grid search for hyperparameter tuning
def grid_search():
    best_rmse_val = float("inf")
    best_params = None

    for param_comb in itertools.product(
        param_space['learning_rate'],
        param_space['max_depth'],
        param_space['num_leaves'],
        param_space['n_estimators']
    ):
        # Prepare parameter dictionary
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': param_comb[0],
            'max_depth': param_comb[1],
            'num_leaves': param_comb[2],
            'n_estimators': param_comb[3],
            'random_state': 42
        }

        # Train model and log metrics
        val_rmse = train_and_log_lightgbm(params)

        # Update best parameters if validation RMSE is lower
        if val_rmse < best_rmse_val:
            best_rmse_val = val_rmse
            best_params = params

    return best_params, best_rmse_val

# Run grid search and log best results
with mlflow.start_run(run_name="Grid Search Hyperparameter Tuning"):
    best_params, best_rmse_val = grid_search()
    mlflow.log_params(best_params)
    mlflow.log_metric("best_rmse_val", best_rmse_val)

print("Best Parameters:", best_params)
print("Best Validation RMSE:", best_rmse_val)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.990961 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69104
[LightGBM] [Info] Number of data points in the train set: 97184, number of used features: 326
[LightGBM] [Info] Start training from score 5.328276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


RMSE Train: 0.27650793016814323, MAPE Train: 246238181231.3274, RMSE Val: 0.274398202717372, MAPE Val: 0.03636539585780591
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.143730 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69104
[LightGBM] [Info] Number of data points in the train set: 97184, number of used features: 326
[LightGBM] [Info] Start training from score 5.328276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


RMSE Train: 0.23820872297283305, MAPE Train: 190867986456.79572, RMSE Val: 0.2627000982070891, MAPE Val: 0.034026932386635414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.979713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69104
[LightGBM] [Info] Number of data points in the train set: 97184, number of used features: 326
[LightGBM] [Info] Start training from score 5.328276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE Train: 0.2194107726138, MAPE Train: 177613238535.9358, RMSE Val: 0.2637395367297886, MAPE Val: 0.03431611957828058
Best Parameters: {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.05, 'max_depth': 7, 'num_leaves': 50, 'n_estimators': 200, 'random_state': 42}
Best Validation RMSE: 0.2608138844684909


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [85]:
best_params

{'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.05,
 'max_depth': 7,
 'num_leaves': 50,
 'n_estimators': 200,
 'random_state': 42}

In [86]:
import joblib
# Start MLflow experiment
mlflow.set_experiment("Final LightGBM Model Training")

with mlflow.start_run(run_name="Final Model Training"):
    # Track training time
    start_time = time.time()

    # Train final model with best parameters
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    model = lgb.train(
        best_params,
        train_data,
        valid_sets=[train_data, val_data]
    )

    training_time = time.time() - start_time

    # Predictions on train, validation, and test sets
    y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
    y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Compute metrics
    rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
    mape_train = mean_absolute_percentage_error(y_train, y_train_pred)

    rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
    mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

    rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
    mape_test = mean_absolute_percentage_error(y_test, y_test_pred)

    # Log parameters and metrics to MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("rmse_train", rmse_train)
    mlflow.log_metric("mape_train", mape_train)
    mlflow.log_metric("rmse_val", rmse_val)
    mlflow.log_metric("mape_val", mape_val)
    mlflow.log_metric("rmse_test", rmse_test)
    mlflow.log_metric("mape_test", mape_test)
    mlflow.log_metric("training_time", training_time)

    # Save model to MLflow
    model_path = "final_lightgbm_model.pkl"
    joblib.dump(model, model_path)
    mlflow.log_artifact(model_path)

    print(f"Training completed - RMSE Train: {rmse_train}, MAPE Train: {mape_train}, RMSE Val: {rmse_val}, MAPE Val: {mape_val}, RMSE Test: {rmse_test}, MAPE Test: {mape_test}")

# Load this model later with joblib.load("final_lightgbm_model.pkl")

2024/11/07 13:06:43 INFO mlflow.tracking.fluent: Experiment with name 'Final LightGBM Model Training' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.916344 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69104
[LightGBM] [Info] Number of data points in the train set: 97184, number of used features: 326
[LightGBM] [Info] Start training from score 5.328276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
